# Dask Bag

## Лабораторная работа 12

In [1]:
import dask.bag as db
import json
import re
import pandas as pd

1. В файлах архиве `reviews_full.zip` находятся файлы, содержащие информацию об отзывах к рецептам в формате JSON Lines. Отзывы разделены на файлы в зависимости от оценки (например, в файле `reviews_1.json` находятся отзывы с оценкой 1). Считайте файлы из этого архива в виде `dask.bag`. Преобразуйте текстовое содержимое файлов в объекты python (с помощью модуля `json`). Выведите на экран первые 5 элементов полученного `bag`.

In [29]:
DATA_GLOB = "reviews_*.json"
rating_re = re.compile(r"reviews_(\d+)\.json$")

In [23]:
# Считывание JSON Lines как dask.bag + предпросмотр
bag_raw = db.read_text(DATA_GLOB, include_path=True)     # (line, path)
bag = bag_raw.map(lambda lp: json.loads(lp[0]))
print("Первые 5 элементов:", bag.take(5))

Первые 5 элементов: ({'user_id': 452355, 'recipe_id': 292657, 'date': '2016-05-08', 'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.'}, {'user_id': 329304, 'recipe_id': 433404, 'date': '2006-06-14', 'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten'}, {'user_id': 227932, 'recipe_id': 2008187, 'date': '1985-11-19', 'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a perfect combo for us. The sausage gave it nice flavor No guestion , I will be making this often.'}, {'us

2. Модифицируйте функцию разбора JSON таким образом, чтобы в каждый словарь c информацией об отзыве добавить ключ `rating`. Значение получите на основе названия файла (см. аргумент `include_path`), использовав для этого регулярное выражение.

In [31]:
# Парсинг с добавлением rating из имени файла
def parse_with_rating(lp):
    """
    lp = (line, path) из dask.bag с include_path=True
    line — строка JSON, path — полный путь к файлу
    """
    line, path = lp
    obj = json.loads(line)
    # гарантированно найдёт число из имени файла
    m = rating_re.search(path)
    if m:
        obj["rating"] = int(m.group(1))
    else:
        raise ValueError(f"Не удалось извлечь rating из имени файла: {path}")
    return obj

bag_rated = bag_raw.map(parse_with_rating)
print("Первые 5 элементов:", bag_rated.take(5))

Первые 5 элементов: ({'user_id': 452355, 'recipe_id': 292657, 'date': '2016-05-08', 'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.', 'rating': 0}, {'user_id': 329304, 'recipe_id': 433404, 'date': '2006-06-14', 'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten', 'rating': 0}, {'user_id': 227932, 'recipe_id': 2008187, 'date': '1985-11-19', 'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a perfect combo for us. The sausage gave it nice flavor No guestion , I will be 

3. Посчитайте количество отзывов в исходном датасете.

In [33]:
print("Всего отзывов:", bag_rated.count().compute())

Всего отзывов: 9057540


4. Отфильтруйте `bag`, сохранив только отзывы, оставленные в 2014 и 2015 годах.

In [35]:
def is_2014_or_2015(o):
    d = o.get("date","")
    return d.startswith("2014-") or d.startswith("2015-")

bag_f = bag_rated.filter(is_2014_or_2015)
print("Первые 5 элементов:", bag_f.take(5))

Первые 5 элементов: ({'user_id': 229850, 'recipe_id': 1300038, 'date': '2014-10-03', 'review': 'Took this to a New Year&#039;s Eve Party. Everyone loved it! It&#039;s absolutely perfect, the flavor, the crunch, just delicious!', 'rating': 0}, {'user_id': 2706705, 'recipe_id': 133747, 'date': '2015-05-08', 'review': 'Simple and easy way to enjoy a slice of pizza any time!  Well-toasted bread is the key - really toast it!  I put a bit of pizza sauce underneath my cheese for a more pizza-like flavor.  I used sourdough bread & medium cheddar cheese.  Fast & fun!  Great idea!  Made for 1-2-3 Hits Tag Game.', 'rating': 0}, {'user_id': 945545, 'recipe_id': 898468, 'date': '2015-06-30', 'review': 'Delish!  I wanted to make this spicy so I used hot enchilada sauce and jalapeno refried beans.  I forgot to buy the onions so I doctored up the beans with onion powder and granulated garlic.  Added the olives under the cheese and baked, uncovered, for the 25 minutes.  Served with pico de gallo, sour 

5. Выполните препроцессинг отзывов:
    * привести строки к нижнему регистру
    * обрезать пробельные символы в начале и конце строки
    * удалите все символы, кроме английских букв и пробелов
    
Примените препроцессинг ко всем записям из `bag`, полученного в задании 4.

In [39]:
# Препроцессинг текста
non_en = re.compile(r"[^a-zA-Z\s]")
def preprocess(o):
    t = (o.get("review") or "").lower().strip()
    t = non_en.sub(" ", t)
    t = re.sub(r"\s+", " ", t).strip()
    o2 = dict(o); o2["review"] = t
    return o2
bag_p = bag_f.map(preprocess)
print("Первые 5 элементов:", bag_p.take(5))

Первые 5 элементов: ({'user_id': 229850, 'recipe_id': 1300038, 'date': '2014-10-03', 'review': 'took this to a new year s eve party everyone loved it it s absolutely perfect the flavor the crunch just delicious', 'rating': 0}, {'user_id': 2706705, 'recipe_id': 133747, 'date': '2015-05-08', 'review': 'simple and easy way to enjoy a slice of pizza any time well toasted bread is the key really toast it i put a bit of pizza sauce underneath my cheese for a more pizza like flavor i used sourdough bread medium cheddar cheese fast fun great idea made for hits tag game', 'rating': 0}, {'user_id': 945545, 'recipe_id': 898468, 'date': '2015-06-30', 'review': 'delish i wanted to make this spicy so i used hot enchilada sauce and jalapeno refried beans i forgot to buy the onions so i doctored up the beans with onion powder and granulated garlic added the olives under the cheese and baked uncovered for the minutes served with pico de gallo sour cream and avocado chunks fantastic thanks for sharing l

6. Посчитайте количество отзывов в датасете, полученном в результате решения задачи 5. В случае ошибок прокомментируйте результат и исправьте функцию препроцессинга.

In [41]:
# Кол-во после препроцессинга
print("После препроцессинга:", bag_p.count().compute())

После препроцессинга: 735274


7. Посчитайте, как часто в наборе, полученном в задании 5, встречается та или иная оценка

In [43]:
# Частоты оценок
freq = dict(bag_p.pluck("rating").frequencies().compute())
freq_df = pd.DataFrame(sorted(freq.items()), columns=["rating","count"])
print(freq_df.to_string(index=False))

 rating  count
      0  42472
      1   9246
      2   9380
      3  26532
      4 119413
      5 528231


8. Найдите среднее значение `rating` в выборке

In [45]:
# Средняя оценка
avg_rating = (freq_df["rating"]*freq_df["count"]).sum()/freq_df["count"].sum() if len(freq_df)>0 else float("nan")
print("Средняя оценка:", avg_rating)

Средняя оценка: 4.388036296673077


9. Используя метод `foldby`, подсчитать максимальную длину отзывов в зависимости от оценки `rating` в наборе, полученном в задании 5.

In [47]:
# foldby: макс длина отзыва по rating
def key_rating(o): return o["rating"]
def binop_max(acc, o): return max(acc, len(o.get("review","")))
max_len = bag_p.foldby(key=key_rating, binop=binop_max, initial=0, combine=max, combine_initial=0).compute()
max_df = pd.DataFrame(sorted(max_len), columns=["rating","max_review_length"])
print(max_df.to_string(index=False))

 rating  max_review_length
      0               6640
      1               3131
      2               2833
      3               3162
      4               6640
      5               5397
